In [ ]:
import json
import time
import requests
from pathlib import Path
import re
import fitz  # PyMuPDF

INVOKE_URL = "api_key"
SECRET_KEY = "api_key"

# 파일(이미지/페이지)을 bytes 형태로 Clova OCR API에 보내서 OCR 결과 JSON을 받아옴
def _call_clova_from_bytes(
    filename: str,
    file_bytes: bytes,
    fmt: str,
    lang: str = "ko",
    enable_table: bool = True,
    timeout: int = 180,
) -> dict:
    headers = {"X-OCR-SECRET": SECRET_KEY}
    message = {
        "version": "V1",
        "requestId": f"clova-{Path(filename).stem}-{int(time.time()*1000)}",
        "timestamp": int(time.time() * 1000),
        "lang": lang,
        "images": [{"format": fmt, "name": Path(filename).stem}],
        "enableTableDetection": bool(enable_table),
    }
    files = {
        "file": (filename, file_bytes, "application/octet-stream"),
        "message": (None, json.dumps(message), "application/json"),
    }
    r = requests.post(INVOKE_URL, headers=headers, files=files, timeout=timeout)
    r.raise_for_status()
    return r.json()

# PDF를 페이지별로 렌더링해서 PNG 이미지 바이트로 변환
def _pdf_to_png_bytes_list(pdf_path: str, dpi: int = 200) -> list[tuple[str, bytes]]:
    p = Path(pdf_path)
    doc = fitz.open(str(p))
    zoom = dpi / 72
    mat = fitz.Matrix(zoom, zoom)

    out: list[tuple[str, bytes]] = []
    for i in range(len(doc)):
        pix = doc[i].get_pixmap(matrix=mat, alpha=False)
        out.append((f"{p.stem}_p{i+1}.png", pix.tobytes("png")))
    doc.close()
    return out

# Clova가 준 테이블 셀 1개에서 텍스트를 뽑는데, 셀 안에 여러 줄이면 줄바꿈(\n)을 유지해서 합침
def _cell_text_keep_lines(cell: dict) -> str:
    lines: list[str] = []

    if isinstance(cell.get("cellTextLines"), list) and cell["cellTextLines"]:
        for ln in cell["cellTextLines"]:
            if isinstance(ln.get("cellWords"), list) and ln["cellWords"]:
                s = " ".join(
                    [w.get("inferText", "") for w in ln["cellWords"] if w.get("inferText")]
                )
            else:
                s = ln.get("inferText", "")
            s = re.sub(r"\s+", " ", str(s)).strip()
            if s:
                lines.append(s)

    elif isinstance(cell.get("cellWords"), list) and cell["cellWords"]:
        s = " ".join([w.get("inferText", "") for w in cell["cellWords"] if w.get("inferText")])
        s = re.sub(r"\s+", " ", str(s)).strip()
        if s:
            lines.append(s)

    else:
        s = (cell.get("inferText") or cell.get("text") or "")
        s = re.sub(r"\s+", " ", str(s)).strip()
        if s:
            lines.append(s)

    return "\n".join(lines).strip()

# Clova OCR 결과 중에서 페이지의 일반 텍스트 영역(fields)을 전부 이어 붙여 페이지 전체 텍스트로 만듬
def _page_text_from_fields(result_json: dict) -> str:
    img0 = (result_json.get("images") or [{}])[0]
    fields = img0.get("fields") or []
    return " ".join([f.get("inferText", "") for f in fields if f.get("inferText")]).strip()

# 테이블에서 rowSpan / colSpan(병합셀) 정보를 보고, 병합된 셀의 텍스트를 병합 영역 전체 좌표에 복제해서 셀 리스트 만듬
def _merge_cells_with_rowspan(cells: list[dict]) -> list[dict]:
    has_span = any(("rowSpan" in c) or ("colSpan" in c) for c in cells)
    if not has_span:
        return cells

    expanded_cells: list[dict] = []
    for cell in cells:
        row_idx = cell.get("rowIndex", 0)
        col_idx = cell.get("columnIndex", 0)
        row_span = cell.get("rowSpan", 1) or 1
        col_span = cell.get("colSpan", 1) or 1
        text = _cell_text_keep_lines(cell)

        for r in range(row_idx, row_idx + row_span):
            for c in range(col_idx, col_idx + col_span):
                expanded_cells.append(
                    {
                        "rowIndex": r,
                        "columnIndex": c,
                        "text": text,
                        "isMerged": (r != row_idx or c != col_idx),
                    }
                )

    return expanded_cells

# Clova OCR 결과에서 tables를 꺼내서 2차원 배열(grid) 형태로 정리
def _tables_to_schema(result_json: dict) -> dict:
    img0 = (result_json.get("images") or [{}])[0]
    tables = img0.get("tables") or []

    out: dict = {}
    for ti, t in enumerate(tables, start=1):
        cells = t.get("cells") or []
        if not cells:
            out[f"table{ti}"] = {"rows": [], "rowCount": 0, "colCount": 0}
            continue

        has_rc = any("rowIndex" in c for c in cells) and any("columnIndex" in c for c in cells)
        if not has_rc:
            texts = [_cell_text_keep_lines(c) for c in cells]
            texts = [x for x in texts if x]
            rows = [[x] for x in texts]
            out[f"table{ti}"] = {"rows": rows, "rowCount": len(rows), "colCount": 1 if rows else 0}
            continue

        expanded_cells = _merge_cells_with_rowspan(cells)

        max_r = max(c.get("rowIndex", 0) for c in expanded_cells)
        max_c = max(c.get("columnIndex", 0) for c in expanded_cells)
        grid = [[""] * (max_c + 1) for _ in range(max_r + 1)]

        for c in expanded_cells:
            r = c.get("rowIndex", 0)
            col = c.get("columnIndex", 0)
            text = c.get("text", "")
            if not grid[r][col]:
                grid[r][col] = text

        out[f"table{ti}"] = {
            "rows": grid,
            "rowCount": len(grid),
            "colCount": len(grid[0]) if grid else 0,
        }

    return out

# 전체 파이프라인 엔트리 함수
def clovaOCR(file_path: str, lang: str = "ko", dpi: int = 200, enable_table: bool = True) -> dict:
    p = Path(file_path)
    if not p.exists():
        raise FileNotFoundError(str(p.resolve()))

    pages_out: dict = {}

    if p.suffix.lower() == ".pdf":
        page_pngs = _pdf_to_png_bytes_list(str(p), dpi=dpi)

        for i, (fname, png_bytes) in enumerate(page_pngs, start=1):
            print(f"[{i}/{len(page_pngs)}] OCR...") # OCR 되는지 확인 용 디버깅
            rj = _call_clova_from_bytes(
                fname, png_bytes, fmt="png", lang=lang, enable_table=enable_table
            )
            tables = _tables_to_schema(rj)

            pages_out[f"page{i}"] = {
                "text": _page_text_from_fields(rj),
                "tableCount": len(tables),
                "tables": tables,
            }

        return {"pageCount": len(page_pngs), "pages": pages_out}

    with open(p, "rb") as f:
        b = f.read()

    fmt = p.suffix.lower().lstrip(".")
    if fmt == "jpeg":
        fmt = "jpg"

    rj = _call_clova_from_bytes(p.name, b, fmt=fmt, lang=lang, enable_table=enable_table)
    tables = _tables_to_schema(rj)

    pages_out["page1"] = {
        "text": _page_text_from_fields(rj),
        "tableCount": len(tables),
        "tables": tables,
    }
    return {"pageCount": 1, "pages": pages_out}

In [ ]:
result = clovaOCR("./ESG 종합 보고서 예시.pdf", lang="ko", dpi=200, enable_table=True)

[1/1] OCR...
{'pageCount': 1, 'pages': {'page1': {'text': '11. 정보보안 영역에서 보안사고 대응 및 백업 정책이 미충족이며, 백업 주기 데이터도 누락되어 운영중단·데이터 손실 리스크가 큼. 12. 정량 항목 25 개 중 누락 10 개, 불확실 8개로 데이터 완성도가 낮아, 등급 산정 시 보수적으로(리스크 상향) 반영됨. 1.3 핵심 리스크 요인 Top 5 순위 리스크 요인 근거(연관 항목) 영향도 긴급도 1 보안사고 대응 및 정성-정보보안(미충족), 매우 높음 높음 백업정책부재 정량 QN-22(누락) 2 산업재해/아차사고 정성-안전보건(미충족), 높음 높음 재발방지 체계 미흡 정량 QN-03(누락) 3 익명신고채널·보호 정성-인권(미충족), 정량 높음 중간 절차 미흡 QN-17(누락) 4 온실가스 배출량 정량 QN-07(누락), 중간 중간 산정 데이터 누락 에너지 모니터링(부분충족) 5 2 차 협력사 ESG 정성- 중간 중간 평가/개선 추적 미흡 공급망(부분충족/미확인), 정량 QN-19~20(누락) 1.4 개선 권고(피드백) 권고ID 권고 내용 관련 리스크 우선순위 권장 담당(예시) 기한 R-01 익명 신고채널(웹/전화) 구축 및 인권/윤리 최상 30 일 현대협력 신고자 보호 절차(비밀보장·불이익 금지·처리기한) 문서화 R-02 산업재해·아차사고 보고-조사- 안전보건 최상 45 일 현대협력 재발방지 표준 프로세스 수립(원인분석, CAPA, 재발검증 포함) R-03 백업정책 정보보안 최상 45 일 현대협력 수립(대상/주기/보관/복구 테스트) 및 랜섬웨어 대응 4', 'tableCount': 2, 'tables': {'table1': {'rows': [['순위', '리스크 요인', '근거(연관 항목)', '영향도', '긴급도'], ['1', '보안사고 대응 및\n백업정책부재', '정성-정보보안(미충족),\n정량 QN-22(누락)', '매우 높음', '높음'], ['2', '산업재해/아차사고\n재발방지 체계 미흡',

In [13]:
result

{'pageCount': 1,
 'pages': {'page1': {'text': '11. 정보보안 영역에서 보안사고 대응 및 백업 정책이 미충족이며, 백업 주기 데이터도 누락되어 운영중단·데이터 손실 리스크가 큼. 12. 정량 항목 25 개 중 누락 10 개, 불확실 8개로 데이터 완성도가 낮아, 등급 산정 시 보수적으로(리스크 상향) 반영됨. 1.3 핵심 리스크 요인 Top 5 순위 리스크 요인 근거(연관 항목) 영향도 긴급도 1 보안사고 대응 및 정성-정보보안(미충족), 매우 높음 높음 백업정책부재 정량 QN-22(누락) 2 산업재해/아차사고 정성-안전보건(미충족), 높음 높음 재발방지 체계 미흡 정량 QN-03(누락) 3 익명신고채널·보호 정성-인권(미충족), 정량 높음 중간 절차 미흡 QN-17(누락) 4 온실가스 배출량 정량 QN-07(누락), 중간 중간 산정 데이터 누락 에너지 모니터링(부분충족) 5 2 차 협력사 ESG 정성- 중간 중간 평가/개선 추적 미흡 공급망(부분충족/미확인), 정량 QN-19~20(누락) 1.4 개선 권고(피드백) 권고ID 권고 내용 관련 리스크 우선순위 권장 담당(예시) 기한 R-01 익명 신고채널(웹/전화) 구축 및 인권/윤리 최상 30 일 현대협력 신고자 보호 절차(비밀보장·불이익 금지·처리기한) 문서화 R-02 산업재해·아차사고 보고-조사- 안전보건 최상 45 일 현대협력 재발방지 표준 프로세스 수립(원인분석, CAPA, 재발검증 포함) R-03 백업정책 정보보안 최상 45 일 현대협력 수립(대상/주기/보관/복구 테스트) 및 랜섬웨어 대응 4',
   'tableCount': 2,
   'tables': {'table1': {'rows': [['순위', '리스크 요인', '근거(연관 항목)', '영향도', '긴급도'],
      ['1', '보안사고 대응 및\n백업정책부재', '정성-정보보안(미충족),\n정량 QN-22(누락)', '매우 높음', '높음'],
      ['2',
       '산업재해/아차사고\

In [12]:
result.keys()

dict_keys(['pageCount', 'pages'])

In [10]:
import pandas as pd

pd.DataFrame(result["pages"]["page1"]['tables']['table1'])

,rows,rowCount,colCount
0,"[순위, 리스크 요인, 근거(연관 항목), 영향도, 긴급도]",6,5
1,"[1, 보안사고 대응 및\n백업정책부재, 정성-정보보안(미충족),\n정량 QN-22...",6,5
2,"[2, 산업재해/아차사고\n재발방지 체계 미흡, 정성-안전보건(미충족),\n정량 Q...",6,5
3,"[3, 익명신고채널·보호\n절차 미흡, 정성-인권(미충족), 정량\nQN-17(누락...",6,5
4,"[4, 온실가스 배출량\n산정 데이터 누락, 정량 QN-07(누락),\n에너지\n모...",6,5
5,"[5, 2 차 협력사 ESG\n평가/개선 추적 미흡, 정성-\n공급망(부분충족/미확...",6,5


In [11]:
pd.DataFrame(result["pages"]["page1"]['tables']['table2'])

,rows,rowCount,colCount
0,"[권고ID, 권고 내용, 관련 리스크, 우선순위, 권장\n기한, 담당(예시)]",8,6
1,"[R-01, 익명 신고채널(웹/전화) 구축 및\n신고자 보호, 인권/윤리, 최상, ...",8,6
2,"[R-01, 절차(비밀보장·불이익, 인권/윤리, 최상, 30 일, 현대협력]",8,6
3,"[R-01, 금지·처리기한) 문서화, 인권/윤리, 최상, 30 일, 현대협력]",8,6
4,"[R-02, 산업재해·아차사고 보고-조사-\n재발방지 표준 프로세스, 안전보건, 최...",8,6
5,"[R-02, 수립(원인분석, CAPA, 재발검증\n포함), 안전보건, 최상, 45 ...",8,6
6,"[R-03, 백업정책, 정보보안, 최상, 45 일, 현대협력]",8,6
7,"[R-03, 수립(대상/주기/보관/복구\n테스트) 및 랜섬웨어 대응, 정보보안, 최...",8,6
